In [2]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd


options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")

driver = webdriver.Chrome(options=options, executable_path='C:/Code/chromedriver/chromedriver.exe')

In [3]:
def get_reviews(driver):
    comments = driver.find_elements_by_xpath('//*[@id="wrap"]/div[3]/yelp-react-root/div/div[3]/div/div/div[2]/div/div[1]/div[2]/section[2]/div[2]/div/ul/li[@*]/div/div[3]/p/span')

    comments_text = []
    for comment in comments:
        comments_text.append(comment.text)

    return comments_text

In [14]:
def extract_comments(driver, num_pages):
    driver.get('https://www.yelp.com/biz/united-airlines-los-angeles-5?sort_by=date_desc');
    time.sleep(3) 
    
    csv = pd.DataFrame(get_reviews(driver))

    for i in range(num_pages-1):
        button = driver.find_element_by_xpath('//*[@id="wrap"]/div[3]/yelp-react-root/div/div[3]/div/div/div[2]/div/div[1]/div[2]/section[2]/div[2]/div/div[4]/div[1]/div/div[11]/span/a')
        button.click()

        time.sleep(2)

        csv_new_page = pd.DataFrame(get_reviews(driver))
        csv = pd.concat([csv, csv_new_page], ignore_index=True)
    
    return csv

In [15]:
df = extract_comments(driver, 5)

In [22]:
df

,0
0,I want to say that I don't know if what these ...
1,12/29/2020 Terrible delay on flight 1887 from ...
2,Shame on you United Airlines! You charged me $...
3,Worst airline experience of my life! I'll neve...
4,I would think in a global Pandemic they would ...
...,...
83,I've had the misfortune of flying United 3 tim...
84,If I could give negative stars it would be app...
85,The flight was delayed for 3 hours and we didn...
86,Avoid at all costs.\nDisgusting company:\nUnhe...


In [23]:
driver.quit()